In [18]:
%run quickstart2.py

Labels:
CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
STARRED
UNREAD


In [85]:
%run access_script.py

Accessed. Yay!
['<div dir="ltr"><p style="border:0px solid rgb(227,227,227);box-sizing:border-box;margin:1.25em 0px;color:rgb(13,13,13);font-family:Söhne,ui-sans-serif,system-ui,-apple-system,&quot;Segoe UI&quot;,Roboto,Ubuntu,Cantarell,&quot;Noto Sans&quot;,sans-serif,&quot;Helvetica Neue&quot;,Arial,&quot;Apple Color Emoji&quot;,&quot;Segoe UI Emoji&quot;,&quot;Segoe UI Symbol&quot;,&quot;Noto Color Emoji&quot;;font-size:16px">Dear [Recipient&#39;s Name],</p><p style="border:0px solid rgb(227,227,227);box-sizing:border-box;margin:1.25em 0px;color:rgb(13,13,13);font-family:Söhne,ui-sans-serif,system-ui,-apple-system,&quot;Segoe UI&quot;,Roboto,Ubuntu,Cantarell,&quot;Noto Sans&quot;,sans-serif,&quot;Helvetica Neue&quot;,Arial,&quot;Apple Color Emoji&quot;,&quot;Segoe UI Emoji&quot;,&quot;Segoe UI Symbol&quot;,&quot;Noto Color Emoji&quot;;font-size:16px">I hope this email finds you well. As we strive to uphold our commitment to environmental sustainability, I am writing to propose sever

In [180]:
import io
import re
import string
import tqdm
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [98]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [91]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print((tokens))

['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


In [92]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [93]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [94]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [95]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

26


In [105]:
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(5, 1): (in, the)
(7, 1): (sun, the)
(7, 6): (sun, hot)
(1, 7): (the, sun)
(2, 1): (wide, the)


In [106]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

In [107]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    # Elements of each training example are appended to these lists.
    targets, contexts, labels = [], [], []

    # Build the sampling table for `vocab_size` tokens.
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    # Iterate over all sequences (sentences) in the dataset.
    for sequence in tqdm.tqdm(sequences):

        # Generate positive skip-gram pairs for a sequence (sentence).
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
                    sequence,
                    vocabulary_size=vocab_size,
                    sampling_table=sampling_table,
                    window_size=window_size,
                    negative_samples=0)

        # Iterate over each positive skip-gram pair to produce training examples
        # with a positive context word and negative samples.
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                    tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                    true_classes=context_class,
                    num_true=1,
                    num_sampled=num_ns,
                    unique=True,
                    range_max=vocab_size,
                    seed=seed,
                    name="negative_sampling")

            # Build context and label vectors (for one target word)
            context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype="int64")

            # Append each element from the training example to global lists.
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)

    return targets, contexts, labels

In [108]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [109]:
with open(path_to_file) as f:
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [110]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [155]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 512
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [156]:
vectorize_layer.adapt(text_ds.batch(1024))

In [157]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [158]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [159]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [160]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████████████████████████████████████████████████████████████████████| 32777/32777 [00:23<00:00, 1411.92it/s]




targets.shape: (24867,)
contexts.shape: (24867, 5)
labels.shape: (24867, 5)


In [163]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [164]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [165]:
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = layers.Embedding(vocab_size,
                                          embedding_dim,
                                          input_length=1,
                                          name="w2v_embedding")
        self.context_embedding = layers.Embedding(vocab_size,
                                               embedding_dim,
                                               input_length=num_ns+1)

    def call(self, pair):
        target, context = pair
        # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
        # context: (batch, context)
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)
        # target: (batch,)
        word_emb = self.target_embedding(target)
        # word_emb: (batch, embed)
        context_emb = self.context_embedding(context)
        # context_emb: (batch, context, embed)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        # dots: (batch, context)
        return dots

In [166]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [167]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [168]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [169]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
24/24 [==============================] - 1s 14ms/step - loss: 1.6065 - accuracy: 0.2921
Epoch 2/20
24/24 [==============================] - 0s 13ms/step - loss: 1.5876 - accuracy: 0.4992
Epoch 3/20
24/24 [==============================] - 0s 14ms/step - loss: 1.5476 - accuracy: 0.5324
Epoch 4/20
24/24 [==============================] - 0s 10ms/step - loss: 1.4862 - accuracy: 0.5136
Epoch 5/20
24/24 [==============================] - 0s 12ms/step - loss: 1.4271 - accuracy: 0.4951
Epoch 6/20
24/24 [==============================] - 0s 10ms/step - loss: 1.3865 - accuracy: 0.4910
Epoch 7/20
24/24 [==============================] - 0s 13ms/step - loss: 1.3571 - accuracy: 0.4990
Epoch 8/20
24/24 [==============================] - 0s 13ms/step - loss: 1.3307 - accuracy: 0.5108
Epoch 9/20
24/24 [==============================] - 0s 13ms/step - loss: 1.3053 - accuracy: 0.5241
Epoch 10/20
24/24 [==============================] - 0s 14ms/step - loss: 1.2802 - accuracy: 0.5358
Epoch 11/

In [170]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [171]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue  # skip 0, it's padding.
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

In [172]:
weights[:10]

array([[ 0.03407152, -0.02948897,  0.02585742, ..., -0.01414828,
         0.00647162, -0.03313811],
       [ 0.03657696, -0.08933467, -0.13787562, ..., -0.0943652 ,
         0.11502133, -0.01733635],
       [-0.08532259,  0.27981082,  0.17016403, ...,  0.11697411,
        -0.00199034,  0.0460463 ],
       ...,
       [ 0.14061852, -0.13591178,  0.2515117 , ..., -0.15124977,
         0.02733884,  0.00854242],
       [ 0.08953336,  0.21459836,  0.11056946, ...,  0.0107081 ,
         0.04126754, -0.06935227],
       [-0.18723059,  0.17448036,  0.17133783, ...,  0.02400639,
         0.14721987, -0.04866942]], dtype=float32)

In [173]:
len(weights)

512

In [174]:
weights.shape

(512, 128)

In [175]:
len(vocab)

512

['',
 '[UNK]',
 'the',
 'and',
 'to',
 'i',
 'of',
 'you',
 'my',
 'a',
 'that',
 'in',
 'is',
 'not',
 'for',
 'with',
 'me',
 'it',
 'be',
 'your',
 'his',
 'this',
 'but',
 'he',
 'have',
 'as',
 'thou',
 'him',
 'so',
 'what',
 'thy',
 'will',
 'no',
 'by',
 'all',
 'king',
 'we',
 'shall',
 'her',
 'if',
 'our',
 'are',
 'do',
 'thee',
 'now',
 'lord',
 'good',
 'on',
 'o',
 'come',
 'from',
 'sir',
 'or',
 'which',
 'more',
 'then',
 'well',
 'at',
 'would',
 'was',
 'they',
 'how',
 'here',
 'she',
 'than',
 'their',
 'them',
 'ill',
 'duke',
 'am',
 'hath',
 'say',
 'let',
 'when',
 'one',
 'go',
 'were',
 'love',
 'may',
 'us',
 'make',
 'upon',
 'yet',
 'richard',
 'like',
 'there',
 'must',
 'should',
 'an',
 'first',
 'why',
 'queen',
 'had',
 'know',
 'man',
 'did',
 'tis',
 'where',
 'see',
 'some',
 'too',
 'death',
 'give',
 'who',
 'these',
 'take',
 'speak',
 'edward',
 'york',
 'mine',
 'such',
 'up',
 'out',
 'henry',
 'romeo',
 'can',
 'father',
 'tell',
 'time',
 

In [151]:
for line in lines:
    w = line.split(" ")
    for word in w:
        if word.strip().lower() in words:
            words[word.strip().lower()] += 1
        else:
            words[word.strip().lower()] = 1

In [152]:
len(words)

27974

In [154]:
words

{'session': 4,
 'First': 235,
 'Citizen:': 98,
 'Before': 31,
 'we': 2338,
 'proceed': 32,
 'any': 545,
 'further,': 12,
 'hear': 562,
 'me': 3347,
 'speak.': 114,
 '': 21723,
 'All:': 19,
 'Speak,': 6,
 'You': 319,
 'are': 2146,
 'all': 2168,
 'resolved': 29,
 'rather': 192,
 'to': 13369,
 'die': 190,
 'than': 1297,
 'famish?': 3,
 'Resolved.': 1,
 'resolved.': 5,
 'First,': 20,
 'you': 7028,
 'know': 857,
 'Caius': 14,
 'Marcius': 30,
 'is': 5694,
 'chief': 18,
 'enemy': 54,
 'the': 17995,
 'people.': 36,
 'We': 182,
 "know't,": 3,
 "know't.": 9,
 'Let': 125,
 'us': 786,
 'kill': 129,
 'him,': 508,
 'and': 14636,
 "we'll": 188,
 'have': 4052,
 'corn': 20,
 'at': 1592,
 'our': 2274,
 'own': 411,
 'price.': 3,
 "Is't": 12,
 'a': 8560,
 'verdict?': 3,
 'No': 94,
 'more': 1313,
 'talking': 12,
 "on't;": 6,
 'let': 1186,
 'it': 3919,
 'be': 4683,
 'done:': 33,
 'away,': 73,
 'away!': 62,
 'Second': 149,
 'One': 45,
 'word,': 69,
 'good': 1704,
 'citizens.': 6,
 'accounted': 6,
 'poor': 42

In [182]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rmgup\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rmgup\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [183]:
sw = stopwords.words('english')

In [184]:
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [187]:
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import Word2Vec as w2v
from sklearn.decomposition import PCA

# constants
PATH = 'shakespeare.txt'
sw = stopwords.words('english')
plt.style.use('ggplot')
# nltk.download('punkt')
# nltk.download('stopwords')

# import data
lines = []
with open(PATH, 'r') as f:
    for l in f:
        lines.append(l)

In [188]:
# remove new lines
lines = [line.rstrip('\n') for line in lines]

# make all characters lower
lines = [line.lower() for line in lines]

# remove punctuations from each line
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]

# tokenize
lines = [word_tokenize(line) for line in lines]

def remove_stopwords(lines, sw = sw):
    '''
    The purpose of this function is to remove stopwords from a given array of 
    lines.
    
    params:
        lines (Array / List) : The list of lines you want to remove the stopwords from
        sw (Set) : The set of stopwords you want to remove
        
    example:
        lines = remove_stopwords(lines = lines, sw = sw)
    '''
    
    res = []
    for line in lines:
        original = line
        line = [w for w in line if w not in sw]
        if len(line) < 1:
            line = original
        res.append(line)
    return res
    
filtered_lines = remove_stopwords(lines = lines, sw = sw)

In [190]:
w = w2v(
    filtered_lines,
    min_count=3,  
    sg = 1,       
    window=7      
)       

print(w.wv.most_similar('lady'))

emb_df = (
    pd.DataFrame(
        [w.wv.get_vector(str(n)) for n in w.wv.key_to_index],
        index = w.wv.key_to_index
    )
)
print(emb_df.shape)
emb_df.head()

[('capulet', 0.8013245463371277), ('anne', 0.7930781245231628), ('grey', 0.790635883808136), ('watchman', 0.7859889268875122), ('third', 0.7754901051521301), ('gentleman', 0.7715964913368225), ('servingman', 0.768513560295105), ('servant', 0.7676213383674622), ('emulation', 0.7654891014099121), ('senator', 0.7646733522415161)]
(4820, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
thou,-0.111388,0.102634,0.311327,0.391375,-0.109722,-0.516248,0.105045,0.652980,-0.378833,-0.461947,...,0.297181,0.083107,0.344467,0.126841,0.428538,0.381706,-0.327570,-0.119157,0.108152,0.194188
thy,-0.156386,0.156076,0.121290,0.073916,0.048472,-0.269125,0.005583,0.590788,-0.183441,-0.304440,...,0.286010,0.042137,0.163752,0.093177,0.256759,0.341058,0.092665,-0.099154,-0.013104,0.055094
king,-0.359416,0.264884,-0.020950,-0.164251,-0.189976,-0.219396,0.034088,0.887656,-0.482256,-0.410423,...,0.349370,-0.124983,0.585505,0.131956,0.677724,0.248554,0.805573,-0.267610,0.403081,-0.457073
shall,-0.036365,0.168755,0.061335,-0.053515,0.018322,-0.152417,0.058797,0.487652,-0.114386,-0.189049,...,0.327524,-0.033843,0.078596,0.011499,0.326723,0.264412,0.164724,-0.066832,-0.106096,0.040973
thee,0.008642,0.129900,0.075459,0.096226,0.056678,-0.212622,0.051666,0.499368,-0.125888,-0.255334,...,0.369448,0.078539,0.043195,0.106807,0.279182,0.371100,-0.035646,-0.009103,-0.142610,0.125969


In [191]:


info = api.info()

In [193]:
model = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [195]:
type(model)

gensim.models.keyedvectors.KeyedVectors

In [198]:
model["hello"]

array([-0.77069  ,  0.12827  ,  0.33137  ,  0.0050893, -0.47605  ,
       -0.50116  ,  1.858    ,  1.0624   , -0.56511  ,  0.13328  ,
       -0.41918  , -0.14195  , -2.8555   , -0.57131  , -0.13418  ,
       -0.44922  ,  0.48591  , -0.6479   , -0.84238  ,  0.61669  ,
       -0.19824  , -0.57967  , -0.65885  ,  0.43928  , -0.50473  ],
      dtype=float32)